In [100]:
#Load Pacakges
library('foreign')
library('dplyr')
library('lme4')
library('nlme')
library(lmerTest)
library(ltm)
library(performance)

In [101]:
#Load Data
dfLong = read.csv('../../Data/daily_dfLong.csv')
dx = read.spss('../../Data/DemosDx/Diagnosis_1.sav',to.data.frame = T)


Warning message in read.spss("../../Data/DemosDx/Diagnosis_1.sav", to.data.frame = T):
“../../Data/DemosDx/Diagnosis_1.sav: Long string value labels record found (record type 7, subtype 21), but ignored”
Warning message in read.spss("../../Data/DemosDx/Diagnosis_1.sav", to.data.frame = T):
“Undeclared level(s) 22 added in variable: eatnos_1”
Warning message in read.spss("../../Data/DemosDx/Diagnosis_1.sav", to.data.frame = T):
“Undeclared level(s) 10 added in variable: addhpr_1”


In [102]:
dx$comorbidity = 
dx[,5:124] %>%
dplyr::select(-contains('nos'))%>%
mutate_all(~ifelse(grepl('Curr',.),1, 0)) %>%
rowSums()

In [103]:
#Find people with MDD as their principle diagnoses
dx_mddPrin = 
dx %>%
filter(grepl('Prin',mddsp_1)|grepl('Prin',mddsnp_1)|grepl('Prin', mddrp_1)|grepl('Prin',mddrnp_1))%>%
mutate(bpd = ifelse(grepl('Curr', bord_1), 1, 0))

In [104]:
#Find most prevalent cormorbid disorders
rank = 
dx_mddPrin[,5:124] %>%
dplyr::select(-contains('nos'))%>%
mutate_all(~ifelse(grepl('Curr',.),1, 0)) %>%
colSums()

dx_rank = data.frame(names(rank),  rank)
dx_rank[order(dx_rank$rank,decreasing = T),]

,names.rank.,rank
,<fct>,<dbl>
mddrnp_1,mddrnp_1,2706
gad4_1,gad4_1,1603
ptsd4_1,ptsd4_1,999
bord_1,bord_1,672
socal4_1,socal4_1,670
mddsnp_1,mddsnp_1,361
dysth_1,dysth_1,352
pda_1,pda_1,339
panic_1,panic_1,299


In [105]:
#Merge daily data with dx
dfLongMDD = merge(dfLong, dx_mddPrin, by = 'ID1',all = F,no.dups = TRUE)

dfLongMDD = dfLongMDD[order(dfLongMDD$ID1, dfLongMDD$day),]
length(unique(dfLongMDD$ID)) #1667


[1] 1667

In [181]:
#Only included patients' records of their first admission
dfLongMDD_firstAdmit = dfLongMDD[is.na(dfLongMDD$Prev_AdmitID_1),]

dfLongMDD = dfLongMDD[is.na(dfLongMDD$Prev_AdmitID_1),]

In [184]:
#First sessino info
first_session = 
dfLongMDD %>%
    group_by(ID1) %>%
    filter(row_number() == 1) %>%
    ungroup()

#First sessino info
first_session_firstAdmit = 
dfLongMDD_firstAdmit %>%
    group_by(ID1) %>%
    filter(row_number() == 1) %>%
    ungroup()

In [185]:
#Examine Treatment Completion Rate
table(first_session$DC_status_1) #Treatment Complete N = 1087
table(first_session$DC_status_1)/nrow(first_session)*100 #65.21%

#Examine Treatment Completion Rate
table(first_session_firstAdmit$DC_status_1) #Treatment Complete N = 689
table(first_session_firstAdmit$DC_status_1)/nrow(first_session_firstAdmit)*100 #66.83%


                                                      0 
                                                      0 
                                                     11 
                                                     14 
                                                     12 
                                                      6 
Inappropriate (language)/Referred to another tx setting 
                                                      9 
                           Insurance no longer covering 
                                                     12 
                                          Nonattendance 
                                                    123 
                                                  Other 
                                                     21 
                                    Other_duplicated_10 
                                                      5 
                 Reported dissatisfaction, withdrew AMA 
                              


                                                      0 
                                              0.0000000 
                                                     11 
                                              1.3579049 
                                                     12 
                                              0.5819593 
Inappropriate (language)/Referred to another tx setting 
                                              0.8729389 
                           Insurance no longer covering 
                                              1.1639185 
                                          Nonattendance 
                                             11.9301649 
                                                  Other 
                                              2.0368574 
                                    Other_duplicated_10 
                                              0.4849661 
                 Reported dissatisfaction, withdrew AMA 
                              


                                                      0 
                                                      0 
                                                     11 
                                                     14 
                                                     12 
                                                      6 
Inappropriate (language)/Referred to another tx setting 
                                                      9 
                           Insurance no longer covering 
                                                     12 
                                          Nonattendance 
                                                    123 
                                                  Other 
                                                     21 
                                    Other_duplicated_10 
                                                      5 
                 Reported dissatisfaction, withdrew AMA 
                              


                                                      0 
                                              0.0000000 
                                                     11 
                                              1.3579049 
                                                     12 
                                              0.5819593 
Inappropriate (language)/Referred to another tx setting 
                                              0.8729389 
                           Insurance no longer covering 
                                              1.1639185 
                                          Nonattendance 
                                             11.9301649 
                                                  Other 
                                              2.0368574 
                                    Other_duplicated_10 
                                              0.4849661 
                 Reported dissatisfaction, withdrew AMA 
                              

In [186]:
#Treatment Completion Rate by BPD
table(first_session$DC_status_1, first_session$bpd)
table(ifelse(first_session$DC_status_1 == 'Treatment Complete','Complete','Incomplete'), first_session$bpd)
summary(glm(ifelse(first_session$DC_status_1 == 'Treatment Complete',1,0) ~first_session$bpd, family = 'binomial'))

                                                         
                                                            0   1
  0                                                         0   0
  11                                                        8   6
  12                                                        4   2
  Inappropriate (language)/Referred to another tx setting   7   2
  Insurance no longer covering                              9   3
  Nonattendance                                            98  25
  Other                                                    20   1
  Other_duplicated_10                                       4   1
  Reported dissatisfaction, withdrew AMA                   29   4
  Transferred to inpatient                                 11   6
  Treatment Complete                                      606  83
  Treatment nearly complete                                30   4
  Withdrew due to external limitations                     59   9

            
               0   1
  Complete   606  83
  Incomplete 279  63


Call:
glm(formula = ifelse(first_session$DC_status_1 == "Treatment Complete", 
    1, 0) ~ first_session$bpd, family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5195  -1.5195   0.8703   0.8703   1.0628  

Coefficients:
                  Estimate Std. Error z value Pr(>|z|)    
(Intercept)        0.77567    0.07235  10.721  < 2e-16 ***
first_session$bpd -0.49996    0.18209  -2.746  0.00604 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1310.2  on 1030  degrees of freedom
Residual deviance: 1302.8  on 1029  degrees of freedom
AIC: 1306.8

Number of Fisher Scoring iterations: 4


In [187]:
#Treatment Completion Rate by # of Comorbidities
summary(glm(ifelse(first_session$DC_status_1 == 'Treatment Complete',1,0) ~ first_session$comorbidity, family = 'binomial'))



Call:
glm(formula = ifelse(first_session$DC_status_1 == "Treatment Complete", 
    1, 0) ~ first_session$comorbidity, family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5416  -1.4615   0.8848   0.9012   1.0210  

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                0.82486    0.14066   5.864 4.51e-09 ***
first_session$comorbidity -0.04452    0.04416  -1.008    0.313    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1310.2  on 1030  degrees of freedom
Residual deviance: 1309.2  on 1029  degrees of freedom
AIC: 1313.2

Number of Fisher Scoring iterations: 4


In [188]:
completeSample = first_session[first_session$DC_status_1 == 'Treatment Complete',]
completeSample

ID1,day,dep1,dep2,dep3,dep4,dep5,dep6,dep7,dep8,⋯,adltadd_1,condct_1,other_1_1,other_1_dx_1,other_2_1,other_2_dx_1,other_3_1,other_3_dx_1,comorbidity,bpd
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>
6,0,3,3,4,1,1,2,3,3,⋯,Not present,Not present,Not present,,Not present,,Not present,,6,0
13,0,3,3,4,1,1,1,1,4,⋯,Not present,Not present,Not present,,Not present,,Not present,,2,0
23,0,4,4,3,1,4,3,5,3,⋯,Not present,Not present,Not present,,Not present,,Not present,,3,0
29,0,4,3,5,1,3,1,3,2,⋯,Not present,Not present,Not present,,Not present,,Not present,,2,0
42,0,5,5,3,2,5,2,3,4,⋯,Not present,Not present,Not present,,Not present,,Not present,,3,0
43,0,3,3,1,1,3,2,3,2,⋯,Not present,Not present,Not present,,Not present,,Not present,,1,0
44,0,4,5,5,5,5,1,4,1,⋯,Not present,Not present,Not present,,Not present,,Not present,,2,0
47,0,1,1,1,1,3,1,1,1,⋯,Not present,Not present,Not present,,Not present,,Not present,,3,0
58,0,5,4,4,1,5,1,5,3,⋯,Not present,Not present,Not present,,Not present,,Not present,,2,0


In [189]:
#How many people were SCIDed
completeSample$SCID_1[completeSample$SCID_1==2] = 'yes'

table(completeSample$SCID_1)

table(completeSample$SCID_1)/sum(table(completeSample$SCID_1))


  2  no yes 
  0 538 151 


        2        no       yes 
0.0000000 0.7808418 0.2191582 

In [148]:
#How many people had comorbid BPD
table(completeSample$bpd)


  0   1 
893 194 

In [149]:
#Comorbidity rates
table(completeSample$comorbidity)
mean(completeSample$comorbidity) #2.75
sd(completeSample$comorbidity) #1.50


  0   1   2   3   4   5   6   7   8   9  10 
  7 160 323 270 181  79  40  19   4   3   1 

[1] 2.922723

[1] 1.49816

In [150]:
rank = 
completeSample[,142:261] %>%
dplyr::select(-contains('nos'))%>%
mutate_all(~ifelse(grepl('Curr',.),1, 0)) %>%
colSums()

dx_rank = data.frame(names(rank),  rank, rank/nrow(completeSample))
dx_rank[order(dx_rank$rank,decreasing = T),][1:10,]

,names.rank.,rank,rank.nrow.completeSample.
,<fct>,<dbl>,<dbl>
mddrnp_1,mddrnp_1,891,0.81968721
gad4_1,gad4_1,479,0.44066237
ptsd4_1,ptsd4_1,283,0.26034959
bord_1,bord_1,194,0.17847286
socal4_1,socal4_1,178,0.16375345
mddsnp_1,mddsnp_1,110,0.10119595
etohab_1,etohab_1,93,0.08555658
panic_1,panic_1,92,0.08463661
pda_1,pda_1,90,0.08279669


In [151]:
#Correlation between BPD and # of cormorbidities 
cor(completeSample$bpd,completeSample$comorbidity) 

[1] 0.3786272

In [152]:
#BPD and Comorbidities vs. Treatment Length
completeSample$tx_length = completeSample$Days_complete_1 + completeSample$Days_not_attended_1
completeSample %>%
group_by(bpd) %>%
summarize(mean = mean(tx_length), sd = sd(tx_length))

t.test(completeSample$tx_length[completeSample$bpd == 0],
      completeSample$tx_length[completeSample$bpd == 1])


completeSample$tx_length = completeSample$Days_complete_1 + completeSample$Days_not_attended_1
completeSample %>%
group_by(bpd) %>%
summarize(mean = mean(comorbidity), sd = sd(comorbidity))

t.test(completeSample$comorbidity[completeSample$bpd == 0],
      completeSample$comorbidity[completeSample$bpd == 1])


#completeSample$tx_attendence_rate = completeSample$Days_complete_1/completeSample$tx_length
#summary(lm(tx_length ~bpd + comorbidity, completeSample)) #Both BPD and Comorbidities predicted longer treatment length in tx completion sample
#summary(lm(tx_attendence_rate ~bpd + comorbidity , completeSample)) #BPD does not predicted attendence rate, while # of comorbidities predicted lower attendence rate

`summarise()` ungrouping output (override with `.groups` argument)



bpd,mean,sd
<dbl>,<dbl>,<dbl>
0,11.23068,5.168298
1,14.45876,11.007809



	Welch Two Sample t-test

data:  completeSample$tx_length[completeSample$bpd == 0] and completeSample$tx_length[completeSample$bpd == 1]
t = -3.9901, df = 211.82, p-value = 9.098e-05
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -4.822836 -1.633323
sample estimates:
mean of x mean of y 
 11.23068  14.45876 


`summarise()` ungrouping output (override with `.groups` argument)



bpd,mean,sd
<dbl>,<dbl>,<dbl>
0,2.658455,1.374344
1,4.139175,1.445454



	Welch Two Sample t-test

data:  completeSample$comorbidity[completeSample$bpd == 0] and completeSample$comorbidity[completeSample$bpd == 1]
t = -13.045, df = 273.97, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -1.704187 -1.257254
sample estimates:
mean of x mean of y 
 2.658455  4.139175 


In [153]:
#Inital Severity
completeSample %>% 
summarize(depM = mean(depMean, na.rm = T), depSD = sd(depMean, na.rm = T),
         anxM = mean(anxMean, na.rm = T), anxSD = sd(anxMean, na.rm = T),
         angM = mean(angMean, na.rm = T), angSD = sd(angMean, na.rm = T))

depM,depSD,anxM,anxSD,angM,angSD
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3.404691,0.7168148,3.1307,0.8878087,2.244165,0.9634773


In [118]:
table(completeSample$Gender_1)
table(completeSample$Gender_1)/nrow(completeSample)
nrow(completeSample)
0.003679853 + 0.005519779 


                                         Female 
                                            720 
                                           Male 
                                            347 
                                     Non-Binary 
                                              4 
                                          Other 
                                              6 
Unknown (ID as Trans Before Demos Change 12.18) 
                                              9 


                                         Female 
                                    0.662373505 
                                           Male 
                                    0.319227231 
                                     Non-Binary 
                                    0.003679853 
                                          Other 
                                    0.005519779 
Unknown (ID as Trans Before Demos Change 12.18) 
                                    0.008279669 

[1] 1087

[1] 0.009199632

In [119]:
mean(completeSample$Age_1)
sd(completeSample$Age_1)
range(completeSample$Age_1)

[1] 40.36523

[1] 14.25956

[1] 18 88

In [120]:
table(completeSample$Race_1)
table(completeSample$Race_1)/nrow(completeSample)



     Asian      Black   Hispanic      Other Portuguese      White 
        15         79        126         45         26        795 


     Asian      Black   Hispanic      Other Portuguese      White 
0.01379945 0.07267709 0.11591536 0.04139834 0.02391904 0.73137075 

### Complete Sample

In [190]:
dfLongMDDComplete = dfLongMDD[dfLongMDD$DC_status_1 == 'Treatment Complete',]
length(unique(dfLongMDDComplete$ID))
dfLongMDDComplete$txlength = dfLongMDDComplete$Days_complete_1 + dfLongMDDComplete$Days_not_attended_1

length(unique(dfLongMDDComplete[dfLongMDDComplete$bpd == 1, 'ID1']))

#ICC, Descriptive Test, t-Tests


model1 = lmer(depMean ~ day + (1|uniquePtId:ID1), data =  dfLongMDDComplete) 
model2 = lmer(dep17 ~ day + (1|uniquePtId:ID1), data =  dfLongMDDComplete) 
model3 = lmer(dep18 ~ day + (1|uniquePtId:ID1), data =  dfLongMDDComplete) 
table1 = data.frame(Var = c('Sym', 'Fun', 'Well-being'),
    ICC = rbind(
    icc(model1, by_group = TRUE),
    icc(model2, by_group = TRUE),
    icc(model3, by_group = TRUE))[,2])

table1


[1] 689

[1] 83

Var,ICC
<fct>,<dbl>
Sym,0.6539684
Fun,0.5778249
Well-being,0.5482682


In [156]:
meanSE = function(x){
    paste0(round(x$coefficient['bpd','Estimate'],2), ' (',
       round(x$coefficient['bpd','Std. Error'],2) ,')')
    
}

In [166]:

#Pre-Treatment Data
dfLongMDDComplete_Session1 = 
    dfLongMDDComplete %>%
    filter(day == 0)

#Pre-Treatment Depressive Symptoms
pre_dep_comp = summary(lmer(depMean ~ bpd + (1|uniquePtI), dfLongMDDComplete_Session1))
pre_dep_comp

#Pre-Treatment Functionality
pre_fun_comp = summary(lmer(dep17 ~ bpd + (1|uniquePtId), dfLongMDDComplete_Session1))
pre_fun_comp

#Pre-Treatment Well-Being
pre_well_comp = summary(lmer(dep18 ~ bpd + (1|uniquePtId), dfLongMDDComplete_Session1))
pre_well_comp

table1 = cbind(table1, c(meanSE(pre_dep_comp), meanSE(pre_fun_comp), meanSE(pre_well_comp)))


#Comorbidity
summary(lmer(comorbidity ~ bpd + (1|uniquePtId), dfLongMDDComplete_Session1))

#Treatment Length
summary(lmer(txlength ~ bpd + (1|uniquePtId), dfLongMDDComplete_Session1))



Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: depMean ~ bpd + (1 | uniquePtId)
   Data: dfLongMDDComplete_Session1

REML criterion at convergence: 2037.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.2707 -0.4691  0.0832  0.6271  2.2671 

Random effects:
 Groups     Name        Variance Std.Dev.
 uniquePtId (Intercept) 0.112    0.3347  
 Residual               0.391    0.6253  
Number of obs: 949, groups:  uniquePtId, 795

Fixed effects:
             Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)   3.37059    0.02605 780.53138 129.367  < 2e-16 ***
bpd           0.19427    0.06130 940.46308   3.169  0.00158 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
    (Intr)
bpd -0.394

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dep17 ~ bpd + (1 | uniquePtId)
   Data: dfLongMDDComplete_Session1

REML criterion at convergence: 2887.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.6966 -0.6394  0.1894  0.9550  1.3206 

Random effects:
 Groups     Name        Variance Std.Dev.
 uniquePtId (Intercept) 0.1571   0.3963  
 Residual               1.1199   1.0582  
Number of obs: 936, groups:  uniquePtId, 786

Fixed effects:
             Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)   3.77150    0.04135 689.62936  91.206   <2e-16 ***
bpd           0.14665    0.09876 878.37203   1.485    0.138    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
    (Intr)
bpd -0.401

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dep18 ~ bpd + (1 | uniquePtId)
   Data: dfLongMDDComplete_Session1

REML criterion at convergence: 2396

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.8654 -0.6152  0.3000  0.4400  1.8631 

Random effects:
 Groups     Name        Variance Std.Dev.
 uniquePtId (Intercept) 0.1140   0.3376  
 Residual               0.6503   0.8064  
Number of obs: 932, groups:  uniquePtId, 783

Fixed effects:
             Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)   3.58304    0.03215 786.01669 111.442   <2e-16 ***
bpd           0.13265    0.07663 910.88426   1.731   0.0838 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
    (Intr)
bpd -0.399

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: comorbidity ~ bpd + (1 | uniquePtId)
   Data: dfLongMDDComplete_Session1

REML criterion at convergence: 3722.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.0536 -0.4424 -0.1150  0.5197  4.8250 

Random effects:
 Groups     Name        Variance Std.Dev.
 uniquePtId (Intercept) 0.3047   0.552   
 Residual               1.6406   1.281   
Number of obs: 1063, groups:  uniquePtId, 877

Fixed effects:
             Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)    2.6719     0.0483  913.4841   55.32   <2e-16 ***
bpd            1.4471     0.1128 1044.7525   12.83   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
    (Intr)
bpd -0.400

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: txlength ~ bpd + (1 | uniquePtId)
   Data: dfLongMDDComplete_Session1

REML criterion at convergence: 7030.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.6469 -0.6169 -0.1791  0.4046 13.9747 

Random effects:
 Groups     Name        Variance Std.Dev.
 uniquePtId (Intercept)  2.991   1.729   
 Residual               40.756   6.384   
Number of obs: 1063, groups:  uniquePtId, 877

Fixed effects:
             Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)   11.2276     0.2266  997.0100   49.56  < 2e-16 ***
bpd            3.0556     0.5333 1041.5395    5.73 1.32e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
    (Intr)
bpd -0.412

In [157]:
table1 = cbind(table1, pre = c(meanSE(pre_dep_comp), meanSE(pre_fun_comp), meanSE(pre_well_comp)))
table1

Var,ICC,pre
<fct>,<dbl>,<fct>
Sym,0.6702102,0.21 (0.06)
Fun,0.5810962,0.13 (0.1)
Well-being,0.5640719,0.11 (0.08)


In [163]:
lmer(depMean ~ bpd + (1|uniquePtId), dfLongMDDComplete_SessionF)


Linear mixed model fit by REML ['lmerModLmerTest']
Formula: depMean ~ bpd + (1 | uniquePtId)
   Data: dfLongMDDComplete_SessionF
REML criterion at convergence: 1653.385
Random effects:
 Groups     Name        Std.Dev.
 uniquePtId (Intercept) 0.4198  
 Residual               0.7466  
Number of obs: 654, groups:  uniquePtId, 574
Fixed Effects:
(Intercept)          bpd  
    2.29659      0.06438  

In [164]:
#Post-Treatment Data
dfLongMDDComplete_SessionF = 
    dfLongMDDComplete %>%
    group_by(ID1) %>%
    slice(day = max(day))%>%
    ungroup() 

#Post-Treatment Depressive Symptoms
post_dep_comp = summary(lmer(depMean ~ bpd + (1|uniquePtId), dfLongMDDComplete_SessionF))
post_dep_comp

#Post-Treatment Functionality
post_fun_comp = summary(lmer(dep17 ~ bpd + (1|uniquePtId), dfLongMDDComplete_SessionF))
post_fun_comp

#Post-Treatment Well-Being
post_well_comp  = summary(lmer(dep18 ~ bpd + (1|uniquePtId), dfLongMDDComplete_SessionF))
post_well_comp 

table1 = cbind(table1, post = c(meanSE(post_dep_comp), meanSE(post_fun_comp), meanSE(post_well_comp)))


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: depMean ~ bpd + (1 | uniquePtId)
   Data: dfLongMDDComplete_SessionF

REML criterion at convergence: 1653.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.6228 -0.6854 -0.1110  0.6003  3.7688 

Random effects:
 Groups     Name        Variance Std.Dev.
 uniquePtId (Intercept) 0.1762   0.4198  
 Residual               0.5575   0.7466  
Number of obs: 654, groups:  uniquePtId, 574

Fixed effects:
             Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)   2.29659    0.03752 603.26514  61.204   <2e-16 ***
bpd           0.06438    0.08878 651.55843   0.725    0.469    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
    (Intr)
bpd -0.394

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dep17 ~ bpd + (1 | uniquePtId)
   Data: dfLongMDDComplete_SessionF

REML criterion at convergence: 1974.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.7239 -0.5987  0.1275  0.9084  2.1860 

Random effects:
 Groups     Name        Variance Std.Dev.
 uniquePtId (Intercept) 0.2634   0.5132  
 Residual               1.0466   1.0231  
Number of obs: 635, groups:  uniquePtId, 560

Fixed effects:
             Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)   2.76658    0.05077 581.47592  54.489   <2e-16 ***
bpd           0.07019    0.12060 632.36424   0.582    0.561    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
    (Intr)
bpd -0.399

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dep18 ~ bpd + +(1 | uniquePtId)
   Data: dfLongMDDComplete_SessionF

REML criterion at convergence: 1680.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9647 -0.8282  0.1024  0.3841  2.2899 

Random effects:
 Groups     Name        Variance Std.Dev.
 uniquePtId (Intercept) 0.2419   0.4918  
 Residual               0.5679   0.7536  
Number of obs: 641, groups:  uniquePtId, 564

Fixed effects:
             Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)   2.88997    0.03998 594.86250  72.292   <2e-16 ***
bpd           0.10485    0.09311 634.35375   1.126    0.261    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
    (Intr)
bpd -0.394

In [126]:
#Change Data
dfLongMDDComplete_Change = 
    dfLongMDDComplete %>%
    group_by(ID1) %>%
    summarize(bpd = bpd[1],
    depMeanC = depMean[n()] - depMean[1],
    dep17C = dep17[n()]-dep17[1],
    dep18C = dep18[n()]-dep18[1],
    SCID_1 = SCID_1[1]) %>%
    ungroup()

#Post-Treatment Depressive Symptoms
change_dep_comp = summary(lm(depMeanC ~ bpd + SCID_1, dfLongMDDComplete_Change))
change_dep_comp

#Post-Treatment Functionality
change_fun_comp = summary(lm(dep17C ~ bpd + SCID_1, dfLongMDDComplete_Change))
change_fun_comp

#Post-Treatment Well-Being
change_well_comp = summary(lm(dep18C ~ bpd + SCID_1, dfLongMDDComplete_Change))
change_well_comp

table1 = cbind(table1, change = c(meanSE(change_dep_comp), meanSE(change_fun_comp), meanSE(change_well_comp)))



`summarise()` ungrouping output (override with `.groups` argument)




Call:
lm(formula = depMeanC ~ bpd + SCID_1, data = dfLongMDDComplete_Change)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.22132 -0.60178 -0.01085  0.65368  2.97961 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -1.16711    0.13938  -8.374 3.41e-16 ***
bpd         -0.02721    0.08649  -0.315    0.753    
SCID_1no     0.01342    0.14200   0.095    0.925    
SCID_1yes   -0.13455    0.16238  -0.829    0.408    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.8516 on 652 degrees of freedom
  (431 observations deleted due to missingness)
Multiple R-squared:  0.00386,	Adjusted R-squared:  -0.0007235 
F-statistic: 0.8422 on 3 and 652 DF,  p-value: 0.4711



Call:
lm(formula = dep17C ~ bpd + SCID_1, data = dfLongMDDComplete_Change)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.9153 -0.8685  0.1315  1.1315  4.2688 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -1.3949     0.2221  -6.280 6.32e-10 ***
bpd          -0.1373     0.1346  -1.020    0.308    
SCID_1no      0.2633     0.2257   1.167    0.244    
SCID_1yes     0.3102     0.2556   1.213    0.225    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.3 on 628 degrees of freedom
  (455 observations deleted due to missingness)
Multiple R-squared:  0.004668,	Adjusted R-squared:  -8.718e-05 
F-statistic: 0.9817 on 3 and 628 DF,  p-value: 0.4009



Call:
lm(formula = dep18C ~ bpd + SCID_1, data = dfLongMDDComplete_Change)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.2316 -0.2316 -0.1562  0.7684  4.7684 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -1.1394     0.1781  -6.398  3.1e-10 ***
bpd          -0.0754     0.1087  -0.693   0.4883    
SCID_1no      0.3710     0.1810   2.050   0.0408 *  
SCID_1yes     0.2955     0.2052   1.440   0.1503    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.042 on 623 degrees of freedom
  (460 observations deleted due to missingness)
Multiple R-squared:  0.008303,	Adjusted R-squared:  0.003527 
F-statistic: 1.739 on 3 and 623 DF,  p-value: 0.1578


In [127]:
table1

Var,ICC,"c(meanSE(pre_dep_comp), meanSE(pre_fun_comp), meanSE(pre_well_comp))",pre,post,change
<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>
Sym,0.6702102,0.21 (0.06),0.21 (0.06),0.07 (0.09),-0.03 (0.09)
Fun,0.5810962,0.13 (0.1),0.13 (0.1),0.08 (0.12),-0.14 (0.13)
Well-being,0.5640719,0.11 (0.08),0.11 (0.08),0.11 (0.09),-0.08 (0.11)


In [129]:
mean_sdc = 
dfLongMDDComplete %>%
group_by(ID1) %>%
summarize(bpd = bpd[1],
    depMeanC = depMean[n()] - depMean[1],
         dep17C = dep17[n()]-dep17[1],
         dep18c = dep18[n()]-dep18[1]) %>%
ungroup()%>%
dplyr::select(-ID1)%>%
group_by(bpd) %>%
summarize_all(mean_sd) %>%
t()

table1 = cbind(table1, mean_sdc[2:4,])
table1



t_test_change_data = 
dfLongMDDComplete %>%
group_by(ID1) %>%
summarize(bpd = bpd[1],
    depMeanC = depMean[n()] - depMean[1],
         dep17C = dep17[n()]-dep17[1],
         dep18c = dep18[n()]-dep18[1]) %>%
ungroup()%>%
dplyr::select(-ID1)

for (i in 1:3){
    print(t.test(t_test_change_data[t_test_change_data$bpd == 0,1+i],
    t_test_change_data[t_test_change_data$bpd == 1,1+i]))
}



`summarise()` ungrouping output (override with `.groups` argument)



ERROR: Error in is_fun_list(.funs): object 'mean_sd' not found


In [130]:
length(unique(dfLongMDDComplete$ID1))

[1] 1087

In [ ]:
dfLongMDDComplete$uniquePtId

In [191]:
summary(lmer(depMean ~ day*bpd + (1+day|ID1) , data = dfLongMDDComplete)) 


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: depMean ~ day * bpd + (1 + day | ID1)
   Data: dfLongMDDComplete

REML criterion at convergence: 9245.7

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.4651 -0.5535 -0.0375  0.4866  7.2267 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 ID1      (Intercept) 0.401597 0.63372       
          day         0.008719 0.09338  -0.02
 Residual             0.178631 0.42265       
Number of obs: 5710, groups:  ID1, 654

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   3.020289   0.028745 648.115675 105.071  < 2e-16 ***
day          -0.127116   0.004843 392.131342 -26.247  < 2e-16 ***
bpd           0.291609   0.082800 635.493747   3.522 0.000459 ***
day:bpd       0.003005   0.013641 356.011840   0.220 0.825761    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
        (Intr) 

In [180]:
summary(lmer(depMean ~ day*bpd + (1+day|ID1) + (1|uniquePtId), data = dfLongMDDComplete)) 


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: depMean ~ day * bpd + (1 + day | ID1) + (1 | uniquePtId)
   Data: dfLongMDDComplete

REML criterion at convergence: 15087.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9453 -0.5559 -0.0297  0.4897  7.2243 

Random effects:
 Groups     Name        Variance Std.Dev. Corr 
 ID1        (Intercept) 0.335706 0.57940       
            day         0.008063 0.08979  -0.03
 uniquePtId (Intercept) 0.101765 0.31901       
 Residual               0.178753 0.42279       
Number of obs: 9366, groups:  ID1, 1021; uniquePtId, 845

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   3.066718   0.025258 869.176260 121.413  < 2e-16 ***
day          -0.114476   0.003836 628.881063 -29.845  < 2e-16 ***
bpd           0.224025   0.059645 987.462470   3.756 0.000183 ***
day:bpd       0.018960   0.008981 556.501840   2.111 0.035214 *  
---
Signif. codes: 

In [175]:


summary(lmer(depMean ~ day*bpd + (1+day|uniquePtId/ID1) , data = dfLongMDDComplete)) 

summary(lmer(depMean ~ day*bpd + (1|uniquePtId)+ (1+day|ID1) , data = dfLongMDDComplete)) 

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: depMean ~ day * bpd + (1 + day | uniquePtId:ID1) + (1 | uniquePtId)
   Data: dfLongMDDComplete

REML criterion at convergence: 15087.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9453 -0.5559 -0.0297  0.4897  7.2243 

Random effects:
 Groups         Name        Variance Std.Dev. Corr 
 uniquePtId:ID1 (Intercept) 0.335701 0.57940       
                day         0.008063 0.08979  -0.03
 uniquePtId     (Intercept) 0.101766 0.31901       
 Residual                   0.178753 0.42279       
Number of obs: 9366, groups:  uniquePtId:ID1, 1021; uniquePtId, 845

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   3.066717   0.025258 869.178292 121.414  < 2e-16 ***
day          -0.114476   0.003836 628.880991 -29.845  < 2e-16 ***
bpd           0.224024   0.059645 987.476183   3.756 0.000183 ***
day:bpd       0.018960   0.008981 556.50132

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.0037338 (tol = 0.002, component 1)”


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: depMean ~ day * bpd + (1 + day | uniquePtId/ID1)
   Data: dfLongMDDComplete

REML criterion at convergence: 15068.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9496 -0.5551 -0.0324  0.4885  7.2266 

Random effects:
 Groups         Name        Variance Std.Dev. Corr 
 ID1:uniquePtId (Intercept) 0.341076 0.58402       
                day         0.004574 0.06763  -0.01
 uniquePtId     (Intercept) 0.096459 0.31058       
                day         0.003602 0.06002  -0.05
 Residual                   0.178646 0.42267       
Number of obs: 9366, groups:  ID1:uniquePtId, 1021; uniquePtId, 845

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   3.068831   0.025240 856.628760 121.586  < 2e-16 ***
day          -0.116441   0.003981 555.367438 -29.251  < 2e-16 ***
bpd           0.228415   0.059661 987.018809   3.829 0.000137 ***
day:bpd   

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: depMean ~ day * bpd + (1 | uniquePtId) + (1 + day | ID1)
   Data: dfLongMDDComplete

REML criterion at convergence: 15087.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9453 -0.5559 -0.0297  0.4897  7.2243 

Random effects:
 Groups     Name        Variance Std.Dev. Corr 
 ID1        (Intercept) 0.335706 0.57940       
            day         0.008063 0.08979  -0.03
 uniquePtId (Intercept) 0.101765 0.31901       
 Residual               0.178753 0.42279       
Number of obs: 9366, groups:  ID1, 1021; uniquePtId, 845

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   3.066718   0.025258 869.176260 121.413  < 2e-16 ***
day          -0.114476   0.003836 628.881063 -29.845  < 2e-16 ***
bpd           0.224025   0.059645 987.462470   3.756 0.000183 ***
day:bpd       0.018960   0.008981 556.501840   2.111 0.035214 *  
---
Signif. codes: 

In [172]:
summary(lmer(depMean ~ day*bpd + (1+day|ID1) , data = dfLongMDDComplete)) 

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: depMean ~ day * bpd + (1 + day | ID1)
   Data: dfLongMDDComplete

REML criterion at convergence: 15097.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9394 -0.5550 -0.0300  0.4883  7.2253 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 ID1      (Intercept) 0.436571 0.66074       
          day         0.008061 0.08978  -0.02
 Residual             0.178750 0.42279       
Number of obs: 9366, groups:  ID1, 1021

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)  3.075e+00  2.454e-02  1.020e+03 125.310  < 2e-16 ***
day         -1.145e-01  3.835e-03  6.287e+02 -29.858  < 2e-16 ***
bpd          2.500e-01  5.915e-02  9.919e+02   4.228 2.58e-05 ***
day:bpd      1.893e-02  8.980e-03  5.563e+02   2.108   0.0355 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
        (Intr

In [137]:
#Predcit changes in depression with bpd while controling for txlength
print('Symptom reduction')
sum_sym = summary(lmer(depMean ~ day*bpd + (1+day|uniquePtId:ID1), data = dfLongMDDComplete)) 
sum_sym$coefficients

print('Functionoing')
sum_fun = summary(lmer(dep17 ~ day*bpd + (1+day|uniquePtId:ID1), data = dfLongMDDComplete))
sum_fun$coefficients

print('Wellbeing')
sum_well = summary(lmer(dep18 ~ day*bpd +   (1+day|uniquePtId:ID1), data = dfLongMDDComplete))
sum_well$coefficients

[1] "Symptom reduction"


,Estimate,Std. Error,df,t value,Pr(>|t|)
(Intercept),3.07511357,0.024540032,1020.3415,125.310086,0.000000e+00
day,-0.11451390,0.003835257,628.7097,-29.858208,1.192343e-122
bpd,0.25004494,0.059145110,991.9500,4.227652,2.579453e-05
day:bpd,0.01893208,0.008980405,556.3372,2.108155,3.546419e-02


[1] "Functionoing"


,Estimate,Std. Error,df,t value,Pr(>|t|)
(Intercept),3.56282374,0.035065679,1009.8757,101.604298,0.000000e+00
day,-0.10150232,0.004663468,580.8606,-21.765416,2.932625e-77
bpd,0.19591217,0.084190880,972.8106,2.327000,2.016977e-02
day:bpd,0.01605806,0.010604361,475.3005,1.514288,1.306175e-01


[1] "Wellbeing"


,Estimate,Std. Error,df,t value,Pr(>|t|)
(Intercept),3.45545714,0.026409371,1016.3806,130.842082,0.000000e+00
day,-0.06800142,0.003470658,548.6479,-19.593236,3.359759e-65
bpd,0.16070578,0.063340496,974.5428,2.537173,1.133013e-02
day:bpd,0.01126395,0.007845611,440.0637,1.435701,1.517979e-01


In [26]:
#Chronbach's Alpha
cronbach.alpha(dfLongMDDComplete[dfLongMDDComplete$day == 0,grep('^dep[0-9]*$', colnames(dfLongMDDComplete))], na.rm = T)



Cronbach's alpha for the 'dfLongMDDComplete[dfLongMDDComplete$day == 0, grep("^dep[0-9]*$", ' '    colnames(dfLongMDDComplete))]' data-set

Items: 18
Sample units: 1063
alpha: 0.855


In [168]:
sum_dep_co

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: depMean ~ day * (bpd + comorbidity) + (1 + day | uniquePtId:ID1)
   Data: dfLongMDDComplete

REML criterion at convergence: 15073.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9431 -0.5556 -0.0294  0.4878  7.2246 

Random effects:
 Groups         Name        Variance Std.Dev. Corr 
 uniquePtId:ID1 (Intercept) 0.424667 0.65167       
                day         0.007937 0.08909  -0.04
 Residual                   0.178794 0.42284       
Number of obs: 9366, groups:  uniquePtId:ID1, 1021

Fixed effects:
                  Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)      2.865e+00  4.911e-02  1.023e+03  58.339  < 2e-16 ***
day             -1.354e-01  7.732e-03  6.300e+02 -17.513  < 2e-16 ***
bpd              1.340e-01  6.301e-02  9.961e+02   2.127  0.03363 *  
comorbidity      7.860e-02  1.595e-02  1.013e+03   4.928 9.72e-07 ***
day:bpd          7.591e-03  9

In [167]:
#Predcit changes in depression with bpd and comorbidities while controling for txlength

print('Symptom reduction')
sum_dep_co = summary(lmer(depMean ~ day*(bpd+comorbidity) +  (1+day|uniquePtId:ID1), data = dfLongMDDComplete)) 
sum_dep_co$coefficients

print('Functionoing')
sum_fun_co = summary(lmer(dep17 ~ day*(bpd+comorbidity) +  (1+day|uniquePtId:ID1), data = dfLongMDDComplete))
sum_fun_co$coefficients

print('Wellbeing')
sum_well_co = summary(lmer(dep18 ~ day*(bpd+comorbidity) +   (1+day|uniquePtId:ID1), data = dfLongMDDComplete))
sum_well_co$coefficients

[1] "Symptom reduction"


,Estimate,Std. Error,df,t value,Pr(>|t|)
(Intercept),2.865287171,0.049114334,1022.9923,58.3391225,0.000000e+00
day,-0.135410397,0.007731774,629.9646,-17.5134966,3.009401e-56
bpd,0.134039601,0.063006045,996.1319,2.1274086,3.363128e-02
comorbidity,0.078599122,0.015950752,1013.2145,4.9276124,9.718160e-07
day:bpd,0.007591224,0.009610878,555.0635,0.7898575,4.299483e-01
day:comorbidity,0.007738217,0.002450587,589.8061,3.1576995,1.671437e-03


[1] "Functionoing"


,Estimate,Std. Error,df,t value,Pr(>|t|)
(Intercept),3.241696989,0.070069025,1012.6200,46.2643368,5.383846e-252
day,-0.108641457,0.009463033,581.4676,-11.4806170,1.219522e-27
bpd,0.018521969,0.089525001,977.5122,0.2068916,8.361376e-01
comorbidity,0.120224771,0.022713707,998.3417,5.2930493,1.478993e-07
day:bpd,0.012541338,0.011406221,476.8030,1.0995174,2.720973e-01
day:comorbidity,0.002551908,0.002954899,528.2890,0.8636192,3.881890e-01


[1] "Wellbeing"


,Estimate,Std. Error,df,t value,Pr(>|t|)
(Intercept),3.300362559,0.053162585,1018.1921,62.080551,0.000000e+00
day,-0.082698714,0.007024756,546.9237,-11.772467,1.129726e-28
bpd,0.074692419,0.067863635,979.4454,1.100625,2.713303e-01
comorbidity,0.058263226,0.017226504,1002.1931,3.382185,7.469147e-04
day:bpd,0.003689842,0.008411469,441.2925,0.438668,6.611168e-01
day:comorbidity,0.005327018,0.002187008,492.3934,2.435756,1.521442e-02


In [28]:
#Interaction between bpd and txlength
print('Symptom reduction')
summary(lmer(depMean ~ (day*txlength)* bpd +  (1+day| ID1), data = dfLongMDDComplete)) 

#print('Functionoing')
#summary(lmer(dep17 ~ day*(bpd+comorbidity) + txlength +  (1+day| ID1), data = dfLongMDDComplete))

#print('Wellbeing')
#summary(lmer(dep18 ~ day*(bpd+comorbidity) + txlength +  (1+day| ID1), data = dfLongMDDComplete))

[1] "Symptom reduction"


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: depMean ~ (day * txlength) * bpd + (1 + day | ID1)
   Data: dfLongMDDComplete

REML criterion at convergence: 14774.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9508 -0.5532 -0.0280  0.4913  7.2047 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 ID1      (Intercept) 0.414966 0.64418       
          day         0.005618 0.07495  -0.17
 Residual             0.178209 0.42215       
Number of obs: 9366, groups:  ID1, 1021

Fixed effects:
                   Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)       2.752e+00  5.760e-02  1.017e+03  47.775  < 2e-16 ***
day              -2.390e-01  8.826e-03  7.784e+02 -27.080  < 2e-16 ***
txlength          3.068e-02  4.591e-03  9.550e+02   6.683 3.97e-11 ***
bpd               4.310e-01  1.029e-01  9.824e+02   4.190 3.04e-05 ***
day:txlength      1.011e-02  6.322e-04  5.560e+02  15.988  < 2e-16 ***
da

### Incomplete Sample

In [29]:
#Predcit changes in depression with bpd while controling for txlength

dfLongMDD$txlength = dfLongMDD$Days_complete_1 + dfLongMDD$Days_not_attended_1
dfLongMDD$completion = ifelse(dfLongMDD$DC_status_1 == 'Treatment Complete', 1, 0)

print('Symptom reduction')
summary(lmer(depMean ~ day*bpd *(completion + txlength) +  (1+day| ID1), data = dfLongMDD)) 

print('Functionoing')
summary(lmer(dep17 ~ day*bpd *(completion + txlength) +(1+day| ID1), data = dfLongMDD))

print('Wellbeing')
summary(lmer(dep18 ~ day*bpd *(completion + txlength) +(1+day| ID1), data = dfLongMDD))

[1] "Symptom reduction"


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: depMean ~ day * bpd * (completion + txlength) + (1 + day | ID1)
   Data: dfLongMDD

REML criterion at convergence: 18975.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9249 -0.5414 -0.0174  0.4884  7.1612 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 ID1      (Intercept) 0.42693  0.6534        
          day         0.00573  0.0757   -0.13
 Residual             0.18058  0.4249        
Number of obs: 11724, groups:  ID1, 1522

Fixed effects:
                     Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)         2.999e+00  4.979e-02  1.626e+03  60.226  < 2e-16 ***
day                -2.143e-01  1.040e-02  1.232e+03 -20.598  < 2e-16 ***
bpd                 3.892e-01  9.051e-02  1.572e+03   4.300 1.82e-05 ***
completion         -1.948e-01  4.651e-02  1.580e+03  -4.188 2.97e-05 ***
txlength            2.589e-02  3.858e-03  1.403e+03   6.7

[1] "Functionoing"


Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00788615 (tol = 0.002, component 1)”


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dep17 ~ day * bpd * (completion + txlength) + (1 + day | ID1)
   Data: dfLongMDD

REML criterion at convergence: 29085.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.9999 -0.5636  0.0115  0.5441  4.2598 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 ID1      (Intercept) 0.80878  0.89932       
          day         0.00821  0.09061  -0.33
 Residual             0.49274  0.70196       
Number of obs: 11480, groups:  ID1, 1516

Fixed effects:
                     Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)         3.325e+00  7.137e-02  1.632e+03  46.584  < 2e-16 ***
day                -1.886e-01  1.429e-02  1.119e+03 -13.200  < 2e-16 ***
bpd                 4.793e-01  1.290e-01  1.559e+03   3.716 0.000209 ***
completion         -2.694e-01  6.646e-02  1.579e+03  -4.053 5.30e-05 ***
txlength            4.803e-02  5.455e-03  1.348e+03   8.805

[1] "Wellbeing"


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dep18 ~ day * bpd * (completion + txlength) + (1 + day | ID1)
   Data: dfLongMDD

REML criterion at convergence: 23693.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.0097 -0.5459 -0.0074  0.5262  4.4242 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 ID1      (Intercept) 0.456953 0.67598       
          day         0.004198 0.06479  -0.29
 Residual             0.316486 0.56257       
Number of obs: 11453, groups:  ID1, 1516

Fixed effects:
                     Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)         3.275e+00  5.444e-02  1.636e+03  60.152  < 2e-16 ***
day                -1.306e-01  1.087e-02  1.058e+03 -12.015  < 2e-16 ***
bpd                 3.449e-01  9.834e-02  1.560e+03   3.507 0.000465 ***
completion         -2.338e-01  5.064e-02  1.581e+03  -4.616 4.22e-06 ***
txlength            3.928e-02  4.140e-03  1.327e+03   9.487